<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [31]</a>'.</span>

In [1]:
# Parameters
snapshotDir = "weekly_pre_QA\\snapshots\\pre_QA_esri_push\\2021-02-21 07.00.00.838081\\"


julie.tsitron@parks.nyc.gov 2/20/2020

# Imports and Connections to DBs

In [2]:
pwd

'D:\\Projects\\Structures\\scripts'

In [3]:
import pyproj

In [4]:
import geopandas

In [5]:
geopandas.__version__

'0.8.1'

In [6]:
pyproj.__version__

'2.6.1.post1'

In [7]:
# pyproj.

In [8]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame, FeatureLayer, FeatureSet

sys.path.append('../')
from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *

In [9]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [10]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [11]:
con_data_parks = pyodbc.connect('Driver={' + driver + '};Server=' + data_parks_server +
                     ';Database=IPMDB;Trusted_Connection=Yes;')
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                           ';Database=;Trusted_Connection=Yes;')

## Deal with Dates

### m/d/Y H:M:S format:

In [12]:
# structures_dev['COMMISSIONDATE'] = pd.to_datetime(
#     structures_dev['COMMISSIONDATE'],
#     errors='coerce').dt.strftime('%m/%d/%Y %H:%M:%S')

In [13]:
# structures_dev['COMMISSIONDATE']

In [14]:
# structures_dev['FEATURESTATUSCHANGEDATE'] = pd.to_datetime(
#     structures_dev['FEATURESTATUSCHANGEDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [15]:
# structures_dev['RETIREDDATE'] = pd.to_datetime(
#     structures_dev['RETIREDDATE']).dt.strftime('%m/%d/%Y %H:%M:%S')

In [16]:
# structures_dev.columns.values

# Delta Table from structuresdb

In [17]:
# SPATIAL DATASET:
sql_str_deltas = 'select * FROM [structuresdb].[dbo].[tbl_delta_structures]'
struct_deltas = read_geosql(sql_str_deltas,
                            con,
                            geom_raw='shape',
                            geom_col='geometry')

In [18]:
## Still need this ??

struct_deltas.rename(columns={
    'objectid': 'OBJECTID',
    'parks_id': 'SYSTEM',
    'bin': 'BIN',
    'bbl': 'BBL',
    'doitt_id': 'DOITT_ID',
    'ground_elevation': 'Ground_Elevation',
    'height_roof': 'Height_Roof',
    'alteration_year': 'Alteration_Year',
    'construction_year': 'Construction_Year',
    'demolition_year': 'Demolition_Year'
},
                     inplace=True)

In [19]:
struct_deltas.head()

,fid,OBJECTID,SYSTEM,BIN,BBL,DOITT_ID,Ground_Elevation,Height_Roof,Construction_Year,Alteration_Year,Demolition_Year,api_call,doitt_source,date_stamp,geometry
0,2,6218,Q448-ZN01A-BLG0398,4268835,4124060180,919366,19,41.21,1943,1986,None,U,current,2021-02-21 09:02:02.120,"POLYGON ((1048144.032 189855.073, 1048131.716 ..."


In [20]:
# struct_deltas[struct_deltas['SYSTEM']=='Q015-ZN06-BLG0418']

In [21]:
multipolygons = []

for i, row in struct_deltas.iterrows(): 
    if type(row['geometry'])==shapely.geometry.multipolygon.MultiPolygon:
        multipolygons.append(row['SYSTEM'])

if len(multipolygons)!=0:
    ids = '\n'.join(multipolygons)
    mssg = 'SYSTEM ID(s) of building(s) that are MultiPolygons:\n'+ids
    send_email('mycontacts.txt','multipoly_mssg.txt', subject = 'multipolygons', e=mssg)
    
struct_deltas = struct_deltas[~struct_deltas['SYSTEM'].isin(multipolygons)]

In [22]:
multipolygons

[]

## Write Delta Table to geojson file

In [23]:
today = datetime.now().strftime('%Y%m%d')
snapshot = r'C:\\Projects\\Structures\\delta_snapshots/' + today

In [24]:
if not struct_deltas.empty:
    if os.path.exists(snapshot):
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')
    else:
        os.makedirs(snapshot)
        struct_deltas.to_file(snapshot + '/deltas.geojson',
                              encoding='utf-8',
                              driver='GeoJSON')

## Read geojson file to geojson object 

In [25]:
if not struct_deltas.empty:
    with open(snapshot+'/deltas.geojson') as data:
        geojson_deltas = json.load(data)

## Create arcgis featureSet from geojson object

In [26]:
type(struct_deltas)

geopandas.geodataframe.GeoDataFrame

In [27]:
struct_deltas.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: USA - New York - SPCS - Long Island
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [28]:
# fromDF_deltas = FeatureSet.from_dataframe(pd.DataFrame(struct_deltas))

In [29]:
if not struct_deltas.empty:
    fromJSON_deltas = FeatureSet.from_geojson(geojson_deltas)

C:\ProgramData\Anaconda3\envs\structures_vpipm\lib\site-packages\arcgis\features\feature.py:799: UserWarning: crs has been deprecated and will be ignored. Please see: https://tools.ietf.org/html/rfc7946#section-4 for more information.
  warn("crs has been deprecated and will be ignored. Please"+ \


In [30]:
#Check the WKT Projection for the feature set
if not struct_deltas.empty:
    print(fromJSON_deltas.spatial_reference)

{'wkid': 4326}


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [31]:
#Set the WKT projection for each feature in the feature set
if not struct_deltas.empty:
    for f in range(0, len(fromJSON_deltas)):
        fromJSON_deltas.features[f].geometry.update({'spatialReference': {'wkid': 2263}})

NameError: name 'f' is not defined

In [ ]:
#Set the WKT projection for the whole feature set
if not struct_deltas.empty:
    fromJSON_deltas.spatial_reference = {'wkid': 2263}

In [ ]:
#Check the WKT Projection for the feature set
if not struct_deltas.empty:
    print(fromJSON_deltas.spatial_reference)

# Connect to published datasets via GIS object

In [ ]:
gis = GIS(url=portal, verify_cert = False)

# Make Edits

## Structures

### run delta archive stored proc

In [ ]:
crsr = con.cursor()
sqal_stmt = '''
exec [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] 
'''
crsr.execute("{CALL [structuresdb].[dbo].[sp_i_tbl_delta_structures_archive] }")
crsr.commit()
crsr.close()

### Connect to structures feature layer directly

In [ ]:
strct_lyr_url = structures_url

In [ ]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

In [ ]:
structures_features

In [ ]:
len(struct_deltas)

### EDIT

In [ ]:
if not struct_deltas.empty:
    num_updates = len(fromJSON_deltas.features)

In [ ]:
if not struct_deltas.empty:
    try:
        update_result = lyr_structures.edit_features(
            updates=fromJSON_deltas.features)
    except:
        error_mssg = 'Updates were not pushed successfully to Structures. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(num_updates) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_structures_mssg.txt', subject = 'structures push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the Structure layer.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'structures: NO UPDATES to push', e=mssg)

In [ ]:
lyr_structures = FeatureLayer(strct_lyr_url)
structures_features = lyr_structures.query()

## Geosupport

### connect to geosupport service

In [ ]:
geosupport_tbl_url = geosupport_url

In [ ]:
tbl_geosupport = FeatureLayer(geosupport_tbl_url)
geosupport = tbl_geosupport.query()

### Get latest BINs from structures layer

In [ ]:
# structures_valid_BINs = structures_features.sdf[
#     (structures_features.sdf['BIN'] != 0) &
#     (~pd.isnull(structures_features.sdf['BIN']))] 

In [ ]:
# bins = structures_valid_BINs[structures_valid_BINs['BIN'].astype(int) %
#                              100000 != 0]['BIN'].astype(int).to_list()

In [ ]:
# bins = [str(bins[i]) for i in range(0, len(bins))]

In [ ]:
structures_sdf = structures_features.sdf[
    (structures_features.sdf['BIN'] != 0) &
    (~pd.isnull(structures_features.sdf['BIN']))] 

In [ ]:
structs_sdf_final = structures_sdf[(structures_sdf['BIN'].astype(int) % 1000000 != 0) 
                                   &(~pd.isnull(structures_sdf['DOITT_ID']))]

In [ ]:
# doitt_counts = structures_sdf[((~pd.isnull(structures_sdf['DOITT_ID']))
#                                &(pd.isnull(structures_sdf['DOITT_Source'])))][cols].groupby('DOITT_ID').count()

In [ ]:
# doitt_counts[doitt_counts['BIN']>1]

In [ ]:
# structures_sdf.groupby('DOITT_ID').count()[structures_sdf.groupby('DOITT_ID').count()['BIN']>1]

In [ ]:
# cols = ['SYSTEM','BIN','DOITT_ID', 'DOITT_Source']

## DUPLICATE DOITT_ID:
# structures_sdf[structures_sdf['DOITT_ID']==1107938.0]#[cols]

In [ ]:
# structures_sdf[(
#     ((structures_sdf['BIN'].astype(int) % 1000000 != 0) 
#                                    &(~pd.isnull(structures_sdf['DOITT_ID'])))
#                 |((~pd.isnull(structures_sdf['DOITT_ID']))&(~pd.isnull(structures_sdf['DOITT_Source'])))
# )][cols]

In [ ]:
len(structs_sdf_final)

In [ ]:
len(structs_sdf_final['BIN'].unique())

In [ ]:
# DUPLICATE BIN:
# pd.concat(g for _, g in structs_sdf_final.groupby("BIN") if len(g) > 1)

### run master geosupport function

In [ ]:
structs_sdf_final.head()

In [ ]:
print_url()

In [ ]:
df = master_geosupport_func(structs_sdf_final, 'BIN')

In [ ]:
df.head()

### prepare latest geosupport data to be pushed to prod

In [ ]:
#Rename all of the columns from Geosupport so that they map exactly to the schema in SQL Server
dff = (df.rename(columns = {'out_zip_code': 'Zip_Code',
                               'b7sc': 'B7SC',
                               'out_b10sc1': 'B10SC',
                               #'in_boro': 'Boro_Code',
                               'bin': 'BIN',
                               'out_preferred_lgc': 'LGC',
                               'type_meaning': 'Address_Type',
                               'high_address_number': 'High_House_Num',
                               'low_address_number': 'Low_House_Num',
                               'out_sos_ind': 'Street_Side',
                               'street_name': 'Norm_Street_Name',
                               'out_usps_city_name': 'USPS_City',
                               'out_TPAD_bin_status': 'TPAD_BIN_Status',
                               'out_com_dist': 'Community_Board',
                               'out_co': 'City_Council',
                               'out_ad': 'NYS_Assembly',
                               'out_sd': 'NYS_Senate',
                               'out_cd': 'US_Congress',
                               'out_nta': 'NTA_Code',
                               'out_fire_bat': 'Fire_Battalion',
                               'out_fire_co': 'Fire_Co_Num',
                               'out_fire_co_str': 'Fire_Co_Type',
                               'out_fire_div': 'Fire_Division',
                               'out_hurricane_zone': 'HEZ',
                               'out_police_patrol_boro': 'Police_Boro',
                               'Police Patrol Borough Command': 'Police_Boro_Com',
                               'out_police_pct': 'Police_Precinct',
                               'out_san_sched': 'Sanitation_Subsect',
                               'out_san_dist_section': 'Sanitation_District',
                               'out_san_recycle': 'Sanitation_Recycling',
                               'out_san_reg': 'Sanitation_Reg_Pickup',
                               'out_physical_id': 'Physical_ID',
                               'address_id': 'Address_ID'})
       .reindex(columns = ['BIN', 'Boro_Code', 'Address_Type', 'Low_House_Num', 'High_House_Num', 'Norm_Street_Name', 'USPS_City', 
                           'Zip_Code', 'Physical_ID', 'B7SC', 'B10SC', 'LGC', 'Street_Side', 'TPAD_BIN_Status', 'HEZ', 
                           'Community_Board', 'City_Council', 'NYS_Assembly', 'NYS_Senate', 'US_Congress', 'NTA_Code', 
                           'Fire_Battalion', 'Fire_Co_Num', 'Fire_Co_Type', 'Fire_Division', 
                           'Police_Boro', 'Police_Boro_Com', 'Police_Precinct', 
                           'Sanitation_Subsect', 'Sanitation_District', 'Sanitation_Recycling', 'Sanitation_Reg_Pickup', 
                           'Address_ID', 'official_address', 'posted_address'])
       .drop_duplicates())

In [ ]:
len(dff)

In [ ]:
# dff.drop('Sanitation_District',axis=1,inplace=True)

In [ ]:
dff.head(10)

In [ ]:
#If the length of the updated dateframe is greater than 0 then truncate the existing table
if len(dff) > 0:
    crsr = con.cursor()
    crsr.execute('''truncate table structuresdb.dbo.tbl_geosupport_address''')
    crsr.commit()
    crsr.close()

In [ ]:
dff.to_sql('tbl_geosupport_address',
           engine,
           schema='dbo',
           if_exists='append',
           index=False)

In [ ]:
dff.Boro_Code.fillna(0,inplace=True)

In [ ]:
dff.Address_ID.fillna(0,inplace=True)

In [ ]:
dff = dff.drop(columns = ['official_address', 'posted_address','Sanitation_District'])

In [ ]:
geosupport_FSET = FeatureSet.from_dataframe(dff)

In [ ]:
len(geosupport_FSET.features)

### EDIT

In [ ]:
if len(dff) > 0: #len(geosupport_FSET.features):
    try:
        ## TRUNCATE AND REPLACE
        tbl_geosupport.delete_features(where="objectid > 0")
        add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)
    except:
        error_mssg = 'Updates were not pushed successfully to the geosupport table. Please see Notebook output/logs for further details.'
        send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport push: FAILED!', e=error_mssg)
    else:
        success_mssg = 'There were ' + str(len(dff)) + ' updates pushed to Production.'
        send_email('mycontacts.txt','success_geosupport_mssg.txt', subject = 'geosupport push: SUCCESS!', e=success_mssg)
else:
    mssg = 'There were no updates to push to the geosupport table.'
    send_email('mycontacts.txt','fail_update_mssg.txt', subject = 'geosupport: NO UPDATES to push', e=mssg)

In [ ]:
tbl_geosupport.delete_features(where="objectid > 0")
add_to_geosupport = tbl_geosupport.edit_features(adds = geosupport_FSET.features)